In [6]:
import pandas as pd
import numpy as np
import regex as re

In [27]:
alcohol_drink = pd.read_csv('./preprocessed_alcohol.csv')
alcohol_drink

,DrinkName,ModifiedDate,DrinkID,Alcoholic,Category,ImageUrlPath,Glass,IBA,Instruction,Ingredients,Measures
0,'57 Chevy with a White License Plate,2016-07-18 22:49:04,14029,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,Highball glass,NaN,1. Fill a rocks glass with ice 2.add white cre...,"[('Creme de Cacao', 1), ('Vodka', 2)]","[('1 oz white ', 1), ('1 oz ', 2)]"
1,1-900-FUK-MEUP,2016-07-18 22:27:04,15395,Alcoholic,Shot,http://www.thecocktaildb.com/images/media/drin...,Old-fashioned glass,NaN,Shake ingredients in a mixing tin filled with ...,"[('Absolut Kurant', 1), ('Grand Marnier', 2), ...","[('1/2 oz ', 1), ('1/4 oz ', 2), ('1/4 oz ', 3..."
2,110 in the shade,2016-02-03 14:51:57,15423,Alcoholic,Beer,http://www.thecocktaildb.com/images/media/drin...,Beer Glass,NaN,Drop shooter in glass. Fill with beer,"[('Lager', 1), ('Tequila', 2)]","[('16 oz ', 1), ('1.5 oz ', 2)]"
3,151 Florida Bushwacker,2016-07-18 22:28:43,14588,Alcoholic,Milk / Float / Shake,http://www.thecocktaildb.com/images/media/drin...,Beer mug,NaN,Combine all ingredients. Blend until smooth. G...,"[('Malibu rum', 1), ('Light rum', 2), ('151 pr...","[('1/2 oz ', 1), ('1/2 oz ', 2), ('1/2 oz Baca..."
4,155 Belmont,2016-10-05 12:36:28,15346,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,White wine glass,NaN,Blend with ice. Serve in a wine glass. Garnish...,"[('Dark rum', 1), ('Light rum', 2), ('Vodka', ...","[('1 shot ', 1), ('2 shots ', 2), ('1 shot ', ..."
...,...,...,...,...,...,...,...,...,...,...,...
541,Zippy's Revenge,2017-01-28 16:18:33,14065,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,Old-fashioned glass,NaN,Mix Kool-Aid to taste then add Rum and ammaret...,"[('Amaretto', 1), ('Rum', 2), ('Kool-Aid', 3)]","[('2 oz ', 1), ('2 oz ', 2), ('4 oz Grape ', 3)]"
542,Zizi Coin-coin,2017-01-28 16:25:52,14594,Alcoholic,Punch / Party Drink,http://www.thecocktaildb.com/images/media/drin...,Margarita/Coupette glass,NaN,"Pour 5cl of Cointreau on ice, add 2cl of fresh...","[('Cointreau', 1), ('Lemon juice', 2), ('Ice',...","[('5 cl ', 1), ('2 cl ', 2), (' cubes\n', 3), ..."
543,Zoksel,2017-01-28 16:28:50,15691,Alcoholic,Soft Drink / Soda,http://www.thecocktaildb.com/images/media/drin...,Beer pilsner,NaN,"No specific mixinginstructions, just poor ever...","[('Beer', 1), ('Root beer', 2), ('Lemonade', 3...","[(' slice\n', 4)]"
544,Zorbatini,2017-01-28 16:32:35,16963,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,Cocktail glass,NaN,Prepare like a Martini. Garnish with a green o...,"[('Vodka', 1), ('Ouzo', 2)]","[('1 1/4 oz Stoli ', 1), ('1/4 oz ', 2)]"


#### preprocess instruction

remove \r, \n in the text

In [28]:
text = 'Place sugar cube in old fashioned glass and saturate with bitters, add a dash of plain water. Muddle until dissolved.\r\nFill the glass with ice cubes and add whiskey.\r\n\r\nGarnish with orange twist, and a cocktail cherry.'
text = re.sub('\\r\\n', '', text)
print(text)

Place sugar cube in old fashioned glass and saturate with bitters, add a dash of plain water. Muddle until dissolved.Fill the glass with ice cubes and add whiskey.Garnish with orange twist, and a cocktail cherry.


In [39]:
def preprocess_instruction(s):
    s = str(s)
    s = re.sub('\\r\\n', '', s)
    return s
alcohol_drink['Instruction'] = alcohol_drink['Instruction'].apply(preprocess_instruction)

#### Analysis Ingredients and Measures

Ingredients와 Measures를 서로 매칭시킬 필요가 있습니다.

1. 때때로, Ingredients와 Measure의 갯수가 서로 다르게 나타나며, 대게 Ingredients의 갯수가 Measure보다
많지만 가끔 Measure의 수가 더 많을 때가 있습니다.

2. 또한 Measure 데이터는 지저분한 편으로, 공백문자나 반점 등이 많이 포함되어 있습니다.

3. 몇몇 Measure값은 빈 리스트([]) 입니다.

4. 단위 뿐만 아니라 일부 재료를 나타내는 단위가 포함되어 있는 경우도 있습니다.

5. 숫자 앞에 숫자가 붙은 경우 해석이 애매합니다. (ex 1 1/2 oz)

6. 어떤한 단위가 사용되었는지 기록하고, 단위와 숫자를 제외한 문자를 제거하여 정제하도록 합니다.

In [33]:
ingredient_list = alcohol_drink['Ingredients'].to_list()
measure_list = alcohol_drink['Measures'].to_list()

uniq_ingr = np.unique(ingredient_list)
uniq_meas = np.unique(measure_list)


In [56]:
alcohol_drink["Ingredients"][0]

'['

In [53]:
for i in range(len(alcohol_drink)):
    print("Index : ", i)
    ingr_item = alcohol_drink["Ingredients"][i]
    print(ingr_item, len(ingr_item))
    meas_item = alcohol_drink["Measures"][i]
    print(meas_item, len(meas_item))
    print()
    


Index :  0
[('Creme de Cacao', 1), ('Vodka', 2)] 37
[('1 oz white ', 1), ('1 oz ', 2)] 34

Index :  1
[('Absolut Kurant', 1), ('Grand Marnier', 2), ('Chambord raspberry liqueur', 3), ('Midori melon liqueur', 4), ('Malibu rum', 5), ('Amaretto', 6), ('Cranberry juice', 7), ('Pineapple juice', 8)] 193
[('1/2 oz ', 1), ('1/4 oz ', 2), ('1/4 oz ', 3), ('1/4 oz ', 4), ('1/4 oz ', 5), ('1/4 oz ', 6), ('1/2 oz ', 7), ('1/4 oz ', 8)] 128

Index :  2
[('Lager', 1), ('Tequila', 2)] 30
[('16 oz ', 1), ('1.5 oz ', 2)] 31

Index :  3
[('Malibu rum', 1), ('Light rum', 2), ('151 proof rum', 3), ('Dark Creme de Cacao', 4), ('Cointreau', 5), ('Milk', 6), ('Coconut liqueur', 7), ('Vanilla ice-cream', 8)] 168
[('1/2 oz ', 1), ('1/2 oz ', 2), ('1/2 oz Bacardi ', 3), ('1 oz ', 4), ('1 oz ', 5), ('3 oz ', 6), ('1 oz ', 7), ('1 cup ', 8)] 127

Index :  4
[('Dark rum', 1), ('Light rum', 2), ('Vodka', 3), ('Orange juice', 4)] 70
[('1 shot ', 1), ('2 shots ', 2), ('1 shot ', 3), ('1 shot ', 4)] 65

Index :  5
[(